In [1]:
from korbash_lib import PowerMeter, TensionGauge, Motor, MotorSystem, Puller, Everage, Exp_everage, PlotDisplayer, Slider
import thorlabs_apt as apt
import time
import numpy as np
import pandas as pd
import datetime
import PySimpleGUI as sg
from tkinter import messagebox as mb
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import gridplot, layout
from jupyter_ui_poll import ui_events
import ipywidgets
import matplotlib
import scipy
from scipy import stats
import random
import os
output_notebook()
pl = Puller()
random.seed()

# инициализация запускать только один раз

Loading BokehJS ...

[(21, 90113195), (21, 90113196), (21, 90113197)]


In [2]:
pl.ms.StopButton() # кнопки для остоновки моторчиков в нештатной ситуации

GridspecLayout(children=(Button(description='Stop All Motors', layout=Layout(grid_area='widget001'), style=But…

In [22]:
pl.ms.MoveToStart(zapas=10) # сдвиг моторчиков на начальные позиции для растяжки
pl.ms.motorM.MoveTo(15);

In [18]:
pl.tg.SetZeroWeight(15) # калибровка тензодатчика

1.6919013628331623

In [11]:
x0=pl.SetH_avto() # измерение высоты молибдена

SetW finish value= 49.96930123875447
podgon a= 5.354970785433676   b= 49.92835847414547   x0= 14.483130279798464
a= 5.71935076726335   b= 49.402047670304796   x0= 14.637385311463206
data saved


In [12]:
pl.SetW(50, dw=0.1, tau=1); # натяжка волокна

SetW finish value= 49.991737500308425


In [13]:
pl.ms.motorM.MoveTo(pl.ms.motorM.position_max-2); # опускание молибдена вниз перед разогревом

In [14]:
pl.tg.SetZeroWeight(5,50) # калибровка тензодатчика после разогрева молибдена

67.93195673056367

In [23]:
x0=11.6
pl.ms.x0=x0

In [24]:
while pl.ms.IsInMotion():
    pass
dp=PlotDisplayer()
dp.CreateMaket(mainParam='time',power=pl.data[['time','power']],
                    tension=pl.data[['time','tension','tensionWgl', 'tensionEXPgl']],
                  # motors=pl.data[['time','motorR','motorL','motorM']]
                  # v_and_a=pl.data[['time','vL','vR','aL','aR']]
              )
sl=Slider()
sl.NewSl('v',min=0.1,max=9,value=3,step=0.1)
sl.NewSl('a',min=0.1,max=9,value=7,step=0.1)
sl.NewSl('dvKof', max=3,value=0,step=0.01)
sl.NewSl('dv', max=0.5,value=0.001,step=0.0001)
sl.NewSl('window',min=50, max=10000,value=100, step=50)
sl.NewSl('molibdenH',min=-1.6, max=2.6,value=2.6,step=0.01)
sl.NewBtn('molibden','up molibden','doun molibden')
sl.NewBtn('end','end','end2','end3','vse','bolshe ne jmi','nu pravda ne nado', 'samiy umniy da?')
sl.NewBtn('start dv','start dv','stop dv')
sl.NewTt('x')
sl.NewTt('L')
sl.NewTt('R')
sl.NewTt('dv')

In [25]:
pl.ms.MotorsControlStart()
t0 = time.time()
t = t0
dp.Show()
sl.Display()
pl.Clear2()
with ui_events() as poll:
    while True:
        poll(10)
        t = time.time()
        pl.Read()
        x=pl.data['x'].iloc[-1]
        v=sl.Sl['v']()
        a=sl.Sl['a']()
        dv=sl.Sl['dv']()
        ww=int(sl.Sl['window']())
        sl.ChangeValueTt('x',x)
        sl.ChangeValueTt('L',pl.ms.L_x(x))
        sl.ChangeValueTt('R',pl.ms.R_x(x))
        sl.ChangeValueTt('dv',dv)
        if not sl.BtnFl['start dv']:
            dv=0
        if pl.ms.PulMotorsControl(v, a, dv, sl.Sl['molibdenH'](), upFl=sl.BtnFl['molibden'], stFl=sl.BtnFl['end'], dhKof=0.5, ah=9)==-1:
            break
            
        dp.Apdate(for_all=pl.data.iloc[-ww:])
        push_notebook()    
        time.sleep(0.04)
print('ura')
pl.Save()
with ui_events() as poll:
    while sl.BtnFl['end']<3:
        poll(10)
        pl.SetW(2,dw=1,tau=0.5)
print('vse')

xMax= 38.44613234964727 L_x(xMax)= 14.700000000000001   x= -3.69702976854569e-06   L= 5.364049273543316 xEnd= -3.69702976854569e-06 lEnd= 4.721193254764884
ura
data saved


KeyboardInterrupt: 

In [31]:
pl.ms.End() # растановка моторчиков на нули (запускать перед отключением питания)

ending
end


In [32]:
pl.Save() # сохранение данных

data saved
